In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.orm import sessionmaker, declarative_base
import os
from sqlalchemy.orm import declarative_base

In [8]:
train_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas"
val_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija"
test_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas"

In [9]:

Bazine_klase = declarative_base()
rysys_su_baze = create_engine('sqlite:///pomidoru_lapai.db')

Session = sessionmaker(bind=rysys_su_baze)
sesija = Session()

In [10]:
class PomidoraiTrenyravimas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_trenyravimo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)


In [11]:
def irasyti_trenyravimo_paveikslelius(sesija, trenyravimo_kelias):

    for klases_pavadinimas in os.listdir(trenyravimo_kelias):
        klases_kelias = os.path.join(trenyravimo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTrenyravimas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTrenyravimas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti trenyravimo paveiksleiai")

In [12]:
class PomidoraiValidacija(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_validacijos_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_validacijos_paveikslelius(sesija, validacijos_kelias):

    for klases_pavadinimas in os.listdir(validacijos_kelias):
        klases_kelias = os.path.join(validacijos_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiValidacija).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiValidacija(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti validacijos paveiksleiai")

In [13]:
class PomidoraiTestas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_testo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_testo_paveikslelius(sesija, testo_kelias):

    for klases_pavadinimas in os.listdir(test_kelias):
        klases_kelias = os.path.join(test_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTestas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTestas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti testo paveiksleiai")

In [14]:
Bazine_klase.metadata.create_all(rysys_su_baze)

irasyti_trenyravimo_paveikslelius(sesija,train_kelias)
irasyti_validacijos_paveikslelius(sesija,val_kelias)
irasyti_testo_paveikslelius(sesija,test_kelias)

Irasyti trenyravimo paveiksleiai
Irasyti validacijos paveiksleiai
Irasyti testo paveiksleiai


In [15]:

train_df = pd.read_sql_table('Pomidoru_lapai_trenyravimo_duomenys', con=rysys_su_baze)

val_df = pd.read_sql_table('Pomidoru_lapai_validacijos_duomenys', con=rysys_su_baze)

test_df = pd.read_sql_table('Pomidoru_lapai_testo_duomenys', con=rysys_su_baze)


# print(train_df.head())
# print(val_df.head())
# print(test_df.head())

In [ ]:
def issitraukti_paveikslelius(df, dydis=(128, 128)):
    paveiksleliai = []
    klasifikacijos = []

    for indeksas, eilute in df.iterrows():
        paveikslelis = load_img(eilute['kelias'], target_size=dydis)
        paveikslelis_array = img_to_array(paveikslelis)
        paveiksleliai.append(paveikslelis_array)
        klasifikacijos.append(eilute['klases_pavadinimas'])

    return np.array(paveiksleliai), np.array(klasifikacijos)

x_train, y_train = issitraukti_paveikslelius(train_df)
x_val, y_val = issitraukti_paveikslelius(val_df)
x_test, y_test = issitraukti_paveikslelius(test_df)

In [17]:
def uzkoduoti_klases_lable(y_train, y_val, y_test):

    enkoderis = LabelEncoder()
    y_train_skaiciais = enkoderis.fit_transform(y_train)
    y_val_skaiciais = enkoderis.transform(y_val)
    y_test_skaiciais = enkoderis.transform(y_test)

    print("Klases uzkoduotos")

    return y_train_skaiciais,y_val_skaiciais,y_test_skaiciais

y_train, y_val, y_test = uzkoduoti_klases_lable(y_train,y_val,y_test)

Klases uzkoduotos


Duomenu ikelimas mobile net modeliui 

In [101]:
import tensorflow as tf

paveikslelio_dydis = (128, 128)
imties_dydis = 32
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical", 
    shuffle=True,
    seed=SEED)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical",
    shuffle=False,
    seed=SEED)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical",
    shuffle=False,
    seed=SEED)

Found 4500 files belonging to 6 classes.
Found 900 files belonging to 6 classes.
Found 600 files belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


ivesties_forma = (128, 128, 3)
klasiu_skaicius = len(train_ds.class_names)

mobilenet_pagrindas = MobileNetV2(
    input_shape=ivesties_forma,
    include_top=False,
    weights='imagenet')

mobilenet_pagrindas.trainable = False 


modelis = models.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    mobilenet_pagrindas,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(klasiu_skaicius, activation='softmax')])


modelis.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

sustabdymas = EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

history = modelis.fit(train_ds,validation_data=val_ds,epochs=30,callbacks=[sustabdymas])


nuostolis, tikslumas = modelis.evaluate(test_ds, verbose=1)

print(f"Testo nuostolis: {nuostolis:4f}")
print(f"Testo tikslumas: {tikslumas:4f}")